# Cafef

In [3]:
import pandas as pd
import time

In [4]:
import re

text = "Quý 3-2023"

# Regular expression pattern
def get_quarter_year(text):
    pattern = r'Quý\s+(\d+)\s*-\s*(\d{4})'
    match = re.search(pattern, text)
    if match:
        quarter = match.group(1)
        year = match.group(2)
        return quarter, year
    else:
        return None, None


In [5]:
get_quarter_year('Quý  2- 2023')

('2', '2023')

In [6]:
def transpose_data(df, stock_code):
    
    stock_codes = []
    categories = []
    years = []
    quarters = []
    data = []
    
    for row in df.iterrows():
        index, data_row = row
        stock_code = stock_code
        for col in df.columns:
            quarter, year = get_quarter_year(col)
            if quarter is None:
                if col.isdigit():
                    quarter = 0
                    year = col
                else:
                    continue
            stock_codes.append(stock_code)
            categories.append(data_row['category_code'])
            years.append(int(year))
            quarters.append(int(quarter))
            data.append(data_row[col])
    return pd.DataFrame({"stock_code": stock_codes, "category_code": categories, "year": years, "quarter": quarters, "data": data})
    

In [7]:
map_caption_bcdkt_bank = pd.read_csv(r'mapping_data\map_caption_bcdkt_bank.csv')
map_caption_bkqkd_bank = pd.read_csv(r'mapping_data\map_caption_bkqkd_bank.csv')
map_caption_blctt_bank = pd.read_csv(r'mapping_data\map_caption_blctt_bank.csv')

map_caption_bcdkt_non_bank = pd.read_csv(r'mapping_data\map_caption_bcdkt_non_bank.csv')
map_caption_bkqkd_non_bank = pd.read_csv(r'mapping_data\map_caption_bkqkd_non_bank.csv')
map_caption_blctt_non_bank = pd.read_csv(r'mapping_data\map_caption_blctt_non_bank.csv')

map_caption_bcdkt_sec = pd.read_csv(r'mapping_data\map_caption_bcdkt_sec.csv')
map_caption_bkqkd_sec = pd.read_csv(r'mapping_data\map_caption_bkqkd_sec.csv')
map_caption_blctt_sec = pd.read_csv(r'mapping_data\map_caption_blctt_sec.csv')

In [8]:
dict_map_caption_bcdkt_bank = dict(zip(map_caption_bcdkt_bank['vi_caption'], map_caption_bcdkt_bank['category_code']))
dict_map_caption_bkqkd_bank = dict(zip(map_caption_bkqkd_bank['vi_caption'], map_caption_bkqkd_bank['category_code']))
dict_map_caption_blctt_bank = dict(zip(map_caption_blctt_bank['vi_caption'], map_caption_blctt_bank['category_code']))

dict_map_caption_bcdkt_non_bank = dict(zip(map_caption_bcdkt_non_bank['vi_caption'], map_caption_bcdkt_non_bank['category_code']))
dict_map_caption_bkqkd_non_bank = dict(zip(map_caption_bkqkd_non_bank['vi_caption'], map_caption_bkqkd_non_bank['category_code']))
dict_map_caption_blctt_non_bank = dict(zip(map_caption_blctt_non_bank['vi_caption'], map_caption_blctt_non_bank['category_code']))

dict_map_caption_bcdkt_sec = dict(zip(map_caption_bcdkt_sec['vi_caption'], map_caption_bcdkt_sec['category_code']))
dict_map_caption_bkqkd_sec = dict(zip(map_caption_bkqkd_sec['vi_caption'], map_caption_bkqkd_sec['category_code']))
dict_map_caption_blctt_sec = dict(zip(map_caption_blctt_sec['map_24h_caption'], map_caption_blctt_sec['category_code']))

In [9]:
import numpy as np
def insert_null_row(df, index_to_insert):
    null_row = pd.DataFrame([[np.nan]*df.shape[1]], columns=df.columns)
    df_with_null = pd.concat([df.iloc[:index_to_insert], null_row, df.iloc[index_to_insert:]]).reset_index(drop=True)
    return df_with_null


In [10]:
map_caption_bcdkt_bank2 = insert_null_row(map_caption_bcdkt_bank, 0)
map_caption_bcdkt_bank2 = insert_null_row(map_caption_bcdkt_bank2, 0)
map_caption_bcdkt_bank2 = insert_null_row(map_caption_bcdkt_bank2, 0)
map_caption_bcdkt_bank2

,vi_caption,category_code,en_caption
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,"I.Tiền mặt, vàng bạc, đá quý",BS_110,"Cash, precious metals, gemstones"
4,II.Tiền gửi tại Ngân hàng Nhà nước,BS_120,Deposits at the Central Bank
...,...,...,...
80,2.Cam kết trong nghiệp vụ L/C,BS_912,Commitments under L/C operations
81,3.Bảo lãnh khác,BS_913,Other guarantees
82,II.Các cam kết đưa ra,BS_920,Commitments issued
83,1.Cam kết tài trợ cho khách hàng,BS_921,Funding commitments to customers


In [11]:
def map_bank_bsheet_code(df):
    df['category_code'] = map_caption_bcdkt_bank2['category_code']
    df.drop(columns=['category'], inplace=True)
    df.dropna(subset=['category_code'], inplace=True)
    return df

def map_bsheet_code(df, map_df):
    df['category_code'] = map_df['category_code']
    df.drop(columns=['category'], inplace=True)
    df.dropna(subset=['category_code'])
    return df


def map_normal_category_code(df, sheet, is_bank):
    if is_bank:
        if sheet == 'incsta':
            df['category_code'] = df['category'].map(dict_map_caption_bkqkd_bank)
        elif sheet == 'cashflow':
            df['category_code'] = df['category'].map(dict_map_caption_blctt_bank)
    else:
        if sheet == 'incsta':
            df['category_code'] = df['category'].map(dict_map_caption_bkqkd_non_bank)
        elif sheet == 'cashflow':
            df['category_code'] = df['category'].map(dict_map_caption_blctt_non_bank)
    df.drop(columns=['category'], inplace=True)
    df.dropna(subset=['category_code'], inplace=True)
    return df

def map_sec_category_code(df, sheet):
    if sheet == 'bsheet':
        df['category_code'] = df['category'].map(dict_map_caption_bcdkt_sec)
    elif sheet == 'incsta':
        df['category_code'] = df['category'].map(dict_map_caption_bkqkd_sec)
    else:
        df['category_code'] = df['category'].map(dict_map_caption_blctt_sec)
        
    df.drop(columns=['category'], inplace=True)
    df.dropna(subset=['category_code'], inplace=True)
    return df


In [12]:
def chop_number(x):
    x = str(x)
    if len(x) > 16:
        return int(x[:-6])
    return int(x)

def chop_mil(x):
    if abs(x) >=1_000_000:
        return int(x/1_000_000)
    return x

def fix_number_issue(df):
    df['data'] = df['data'].apply(lambda x: str(x).replace(",", ""))
    df['data'] = df['data'].apply(lambda x: chop_number(x))
    df['data'] = df['data'].apply(lambda x: chop_mil(x))
    return df

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# URL of the page
url = 'https://s.cafef.vn/bao-cao-tai-chinh/hpg/incsta/2024/2/0/0/1/ket-qua-hoat-dong-kinh-doanh-.chn'
url = 'https://s.cafef.vn/bao-cao-tai-chinh/hpg/cashflow/2024/2/0/1/luu-chuyen-tien-te-gian-tiep-.chn'
url = 'https://s.cafef.vn/bao-cao-tai-chinh/bid/bsheet/2024/2/0/1/bao-cao-tai-chinh-.chn'
url = 'https://s.cafef.vn/bao-cao-tai-chinh-chung-khoan/vnd/bsheet/2024/2/0/1/bao-cao-tai-chinh-.chn'
url = 'https://s.cafef.vn/bao-cao-tai-chinh-ngan-hang/bid/cashflowdirect/2024/4/0/0/luu-chuyen-tien-te-truc-tiep-.chn'
def craw_cafef(stock_code, start_year, start_month, duration = 2, sheet = 'bsheet', is_bank = False):
# Send a GET request to fetch the raw HTML content
    if start_month == 0:
        start_year += 4
    else:
        start_year += 1
    
    dfs = None
    while duration > 0:
        duration -= 1
        if start_month == 0:
            start_year -= 4
        else:
            start_year -= 1
        
        try:
            if sheet == 'bsheet':
                url = f'https://s.cafef.vn/bao-cao-tai-chinh{"-ngan-hang" if is_bank else ""}/{stock_code.lower()}/{sheet}/{start_year}/{start_month}/0/1/bao-cao-tai-chinh-.chn'
            elif sheet == 'incsta':
                url = f'https://s.cafef.vn/bao-cao-tai-chinh{"-ngan-hang" if is_bank else ""}/{stock_code.lower()}/{sheet}/{start_year}/{start_month}/0/0/ket-qua-hoat-dong-kinh-doanh-.chn'
            elif sheet == 'cashflow':
                url = f'https://s.cafef.vn/bao-cao-tai-chinh{"-ngan-hang" if is_bank else ""}/{stock_code.lower()}/{sheet + "direct" if is_bank else sheet}/{start_year}/{start_month}/0/1/luu-chuyen-tien-te-{"truc" if is_bank else "gian"}-tiep-.chn'
            print(url)
            response = requests.get(url)
            html_content = response.text

            # Parse the content with BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')

            # Find the table elements
            tables = soup.find_all('table') # Adjust as necessary depending on the structure

            row = tables[3].find('tr')
            headers = [cell.text.strip() for cell in row.find_all('td')]
            headers[0] = 'category'
            headers.pop()

            print(headers)

            rows = tables[4].find_all('tr')
            records = []
            for row in rows:
                record = []
                i = 0
                has_text = False
                for cell in row.find_all('td'):
                    if cell.text.strip() != '':
                        has_text = True
                        record.append(cell.text.strip())
                    else:
                        record.append(np.nan)
                    i+=1
                    if i == len(headers):
                        break
                if len(record) > 0 and has_text:
                    records.append(record)
            df = pd.DataFrame(records, columns=headers)
            # return df
            df = df.dropna(how='all', axis=1)
            
            
        #    Map category code
            if sheet == 'bsheet':
                if is_bank:
                    df = map_bsheet_code(df, map_caption_bcdkt_bank2)
                else:
                    df = map_bsheet_code(df, map_caption_bcdkt_non_bank)
                
            else:
                df = map_normal_category_code(df, sheet, is_bank)
                
            df.dropna(subset=['category_code'], inplace=True)
                
            if dfs is None:
                dfs = df
            else:
                dfs = pd.merge(dfs, df, on='category_code', how= 'left', suffixes=('_df1', '_df2'))
                columns_to_check = df.columns
                for col in columns_to_check:
                    if f"{col}_df1" in dfs.columns:
                        dfs[f"{col}"] = dfs[f"{col}_df1"].fillna(dfs[f"{col}_df2"])
                        dfs.drop(columns=[f"{col}_df1",f"{col}_df2"], inplace=True)
        except:
            print(f"Error at {start_year} - {start_month}")
            continue
               
    if dfs is None:
        return None 
      
    
    dfs = transpose_data(dfs, stock_code.upper())
    dfs.dropna(subset=['category_code'], inplace=True)
    dfs.fillna(0, inplace=True)
    dfs['year'] = dfs['year'].astype(int)
    dfs['quarter'] = dfs['quarter'].astype(int)
    return fix_number_issue(dfs)  

In [14]:
dfs =craw_cafef('HPG', 2023, 0, 2, 'bsheet', False)
dfs
#dfs[(dfs['stock_code'] == 'MWG') & (dfs['year'] == 2023) & (dfs['quarter'] == 1)]

https://s.cafef.vn/bao-cao-tai-chinh/hpg/bsheet/2023/0/0/1/bao-cao-tai-chinh-.chn
['category', '2020', '2021', '2022', '2023']
https://s.cafef.vn/bao-cao-tai-chinh/hpg/bsheet/2019/0/0/1/bao-cao-tai-chinh-.chn
['category', '2016', '2017', '2018', '2019']


,stock_code,category_code,year,quarter,data
0,HPG,BS_100,2020,0,56747258
1,HPG,BS_100,2021,0,94154859
2,HPG,BS_100,2022,0,80514708
3,HPG,BS_100,2016,0,18182786
4,HPG,BS_100,2017,0,33068060
...,...,...,...,...,...
814,HPG,BS_440,2022,0,170335519
815,HPG,BS_440,2016,0,33226552
816,HPG,BS_440,2017,0,53022184
817,HPG,BS_440,2018,0,78223007


In [37]:
def craw_24h_financial_statement(stock_code, period, view, pages, lang = 'vi'):
    dfs = None
    for page in range(1, pages+1):
        url = f"https://api-finance-t19.24hmoney.vn/v1/ios/company/financial-report?device_name=INVALID&device_model=Windows+11&network_carrier=INVALID&connection_type=INVALID&os=Chrome&os_version=128.0.0.0&access_token=INVALID&push_token=INVALID&locale={lang}&symbol={stock_code}&period={period}&view={view}&page={page}&expanded=true"
        response = requests.get(url)
        
        if response.status_code == 200:
            try:
                data = response.json()
                headers = data['data']['headers']
                rows = data['data']['rows']
                categories = []
                stock_codes = []
                year = []
                quarter = []
                data = []
                
                for i in range(len(rows)):
                    for j in range(0, len(rows[i]),2):
                        
                        stock_codes.append(stock_code)
                        categories.append(rows[i]['name'])
                        year.append(headers[j]['year'])
                        quarter.append(headers[j]['quarter'])   
                        data.append(rows[i]['values'][j])  
                
                df = pd.DataFrame({ 'stock_code': stock_code, 'category': categories, 'year': year, 'quarter': quarter, 'data': data})
                if dfs is None:
                    dfs = df
                else:
                    dfs = pd.concat([dfs, df]).reset_index(drop=True)
                    
                del df
            
            except:
                print(response.json())
                continue    
            
        else:
            print(response.json()) 
    
    return dfs

In [35]:
def crawl_mix_securities(stock_code, start_year, start_month, duration = 2, sheet = 'bsheet'):
    source = 'cafef'
    if sheet == 'cashflow':
        source = 'money24h'
    
    dfs = None   
    
    if source == 'cafef':
        if start_year == 2024 and start_month == 0:
            start_year = 2023
        while duration > 0:
            if sheet == 'bsheet':
                url = f'https://s.cafef.vn/bao-cao-tai-chinh-chung-khoan/{stock_code.lower()}/{sheet}/{start_year}/{start_month}/0/1/bao-cao-tai-chinh-.chn'
            elif sheet == 'incsta':
                url = f'https://s.cafef.vn/bao-cao-tai-chinh-chung-khoan/{stock_code.lower()}/{sheet}/{start_year}/{start_month}/0/0/ket-qua-hoat-dong-kinh-doanh-.chn'
            response = requests.get(url)
            html_content = response.text

            # Parse the content with BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')
            tables = soup.find_all('table') # Adjust as necessary depending on the structure

            row = tables[3].find('tr')
            headers = [cell.text.strip() for cell in row.find_all('td')]
            headers[0] = 'category'
            headers.pop()

            print(headers)

            rows = tables[4].find_all('tr')
            records = []
            for row in rows:
                record = []
                i = 0
                has_text = False
                for cell in row.find_all('td'):
                    if cell.text.strip() != '':
                        has_text = True
                        record.append(cell.text.strip())
                    else:
                        record.append(np.nan)
                    i+=1
                    if i == len(headers):
                        break
                if len(record) > 0 and has_text:
                    records.append(record)
            df = pd.DataFrame(records, columns=headers)
            df = df.dropna(how='all', axis=1)
            
        #    Map category code
            if sheet == 'bsheet':
                df = map_bsheet_code(df, map_caption_bcdkt_sec)
            else:
                df = map_sec_category_code(df, sheet)
            
            df.dropna(subset=['category_code'], inplace=True)
            
            if dfs is None:
                dfs = df
            else:
                dfs = pd.merge(dfs, df, on='category_code', how= 'left', suffixes=('_df1', '_df2'))
                columns_to_check = df.columns
                for col in columns_to_check:
                    if f"{col}_df1" in dfs.columns:
                        dfs[f"{col}"] = dfs[f"{col}_df1"].fillna(dfs[f"{col}_df2"])
                        dfs.drop(columns=[f"{col}_df1",f"{col}_df2"], inplace=True)    
            del df
                
            duration -= 1
            if start_month == 0:
                start_year -= 4
            else:
                start_year -= 1
        dfs = transpose_data(dfs, stock_code.upper())
        dfs.fillna(0, inplace=True)
        dfs['year'] = dfs['year'].astype(int)
        dfs['quarter'] = dfs['quarter'].astype(int)
        return fix_number_issue(dfs)  
    else:
        period = 1
        if start_month != 0:
            period = 2
            
        view = 3
        
        df = craw_24h_financial_statement(stock_code, period, view, duration)            
        df = map_sec_category_code(df,sheet)
        df['data'] = df['data'].astype(float)
        df['data'] = df['data']*1000
        df.drop_duplicates(subset=['category_code', 'year', 'quarter'], inplace=True)
        df.fillna(0, inplace=True)
        df['data'] = df['data'].astype(int)
        df['year'] = df['year'].astype(int)
        df['quarter'] = df['quarter'].astype(int)
        
        return df
    
        

In [15]:
non_bank_stock_code = ["MWG", "NHA", "VNM", "HPG", "VHM", "PNJ", "YEG", "FPT","MSN", "GAS", "VRE", "VJC", "VIC", "PLX", "SAB", "POW", "GVR", "BCM", "VPI", "DVM", "KDH", "HDC", "TCH", "CEO", "HUT", "NVL", "DBC", "SAF", "DHT", "VTP", "PVT", "FRT", "DGC", "DCM", "NKG", "CMG", "VGI", "PVC", "CAP", "DTD", "HLD", "L14", "L18", "LAS", "LHC", "NTP", "PLC", "PSD", "PVG", "PVS", "SLS", "TIG", "TMB", "TNG", "TVD", "VC3", "VCS", "DXG"]
bank_stock_code = ["BID", "EIB", "OCB", "CTG", "VCB", "ACB", "MBB", "HDB", "TPB", "VPB",  "STB", "TCB",  "SHB", "VIB", "CTG",  "ABB", "LPB", "NVB"]
securities_stock_code = ["MBS", "VND", "SSI", "VIX", "ORS"]

In [2]:
len(non_bank_stock_code) + len(bank_stock_code) + len(securities_stock_code)

81

In [21]:
is_bank = False
sheets = ['incsta', 'bsheet', 'cashflow']
sheet_types = ['income_statement' , 'balance_sheet', 'cash_flow_statement']
dfs = None
start_year = 2024
start_months = [0,2]
durations = [2,6]
count = 1

for sheet, sheet_type in zip(sheets, sheet_types):
    for start_month, duration in zip(start_months, durations):
        for stock_code in non_bank_stock_code:
            if count % 5 == 0:
                time.sleep(5)
            count+=1
            start_year_ = start_year
            if start_month == 0:
                start_year_ -=1
            df = craw_cafef(stock_code, start_year_, start_month, duration, sheet, is_bank)
            if dfs is None:
                dfs = df
            else:
                dfs = pd.concat([dfs, df]).reset_index(drop=True)
            print("Done", stock_code)
            
                

dfs

https://s.cafef.vn/bao-cao-tai-chinh/mwg/incsta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2020', '2021', '2022', '2023']
https://s.cafef.vn/bao-cao-tai-chinh/mwg/incsta/2019/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2016', '2017', '2018', '2019']
Done MWG
https://s.cafef.vn/bao-cao-tai-chinh/nha/incsta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2020', '2021', '2022', '2023']
https://s.cafef.vn/bao-cao-tai-chinh/nha/incsta/2019/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2016', '2017', '2018', '2019']
Done NHA
https://s.cafef.vn/bao-cao-tai-chinh/vnm/incsta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2020', '2021', '2022', '2023']
https://s.cafef.vn/bao-cao-tai-chinh/vnm/incsta/2019/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2016', '2017', '2018', '2019']
Done VNM
https://s.cafef.vn/bao-cao-tai-chinh/hpg/incsta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2020', '2021', '2022', '2023']
https:/

,stock_code,category_code,year,quarter,data
0,MWG,IS_001,2020,0,109801253
1,MWG,IS_001,2016,0,45612676
2,MWG,IS_001,2017,0,67698540
3,MWG,IS_001,2018,0,87738378
4,MWG,IS_001,2019,0,103485046
...,...,...,...,...,...
304376,DXG,CF_070,2020,2,1266336
304377,DXG,CF_070,2018,3,925246
304378,DXG,CF_070,2018,4,1158590
304379,DXG,CF_070,2019,1,1026704


In [22]:
dfs2 = pd.read_csv(r'../csv/non_bank_financial_report_v2_1.csv')
dfs = pd.concat([dfs, dfs2]).reset_index(drop=True)
dfs.drop_duplicates(inplace=True)

In [21]:
assert dfs.duplicated().sum() == 0, "Duplicated rows"

In [23]:
test_df = dfs[(dfs['year'] == 2023) & (dfs['quarter'] == 0)]
assert test_df['stock_code'].nunique() == len(non_bank_stock_code), "Number of stock code is not correct"

AssertionError: Number of stock code is not correct

In [29]:
test_df['stock_code'].unique()

array(['MWG', 'NHA', 'VNM', 'HPG', 'VHM', 'PNJ', 'FPT', 'MSN', 'GAS',
       'VRE', 'VJC', 'SAB', 'POW', 'GVR', 'BCM', 'VPI', 'DVM', 'KDH',
       'HDC', 'TCH', 'CEO', 'HUT', 'NVL', 'DBC', 'SAF', 'DHT', 'VTP',
       'PVT', 'FRT', 'DGC', 'DCM', 'NKG', 'CMG', 'VGI', 'PVC', 'CAP',
       'DTD', 'HLD', 'L14', 'L18', 'LAS', 'LHC', 'PLC', 'PSD', 'PVG',
       'PVS', 'SLS', 'TIG', 'TMB', 'TNG', 'TVD', 'VC3', 'VCS', 'DXG',
       'NTP'], dtype=object)

In [25]:
for code in non_bank_stock_code:
    if  code not in test_df['stock_code'].unique():
        print(f"{code} is missing")

PLX is missing


In [26]:
dfs.fillna(0, inplace=True)
dfs.to_csv('../csv/non_bank_financial_report_v2_1.csv', index=False)

In [27]:
is_bank = True
sheets = ['incsta', 'bsheet', 'cashflow']
sheet_types = ['income_statement' , 'balance_sheet', 'cash_flow_statement']
dfs = None
start_year = 2024
start_months = [0,2]
durations = [2,6]
count = 1

for sheet, sheet_type in zip(sheets, sheet_types):
    for start_month, duration in zip(start_months, durations):
        for stock_code in bank_stock_code:
            if count % 10 == 0:
                time.sleep(5)
            count+=1
            start_year_ = start_year
            if start_month == 0:
                start_year_ -=1
            df = craw_cafef(stock_code, start_year_, start_month, duration, sheet, is_bank)
            if dfs is None:
                dfs = df
            elif df is None:
                continue
            else:
                dfs = pd.concat([dfs, df]).reset_index(drop=True)
            print("Done", stock_code)
            


https://s.cafef.vn/bao-cao-tai-chinh-ngan-hang/bid/incsta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2020', '2021', '2022', '2023']
https://s.cafef.vn/bao-cao-tai-chinh-ngan-hang/bid/incsta/2019/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2016', '2017', '2018', '2019']
Done BID
https://s.cafef.vn/bao-cao-tai-chinh-ngan-hang/eib/incsta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2018', '2019', '2020', '2021']
https://s.cafef.vn/bao-cao-tai-chinh-ngan-hang/eib/incsta/2019/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2016', '2017', '2018', '2019']
Done EIB
https://s.cafef.vn/bao-cao-tai-chinh-ngan-hang/ocb/incsta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2020', '2021', '2022', '2023']
https://s.cafef.vn/bao-cao-tai-chinh-ngan-hang/ocb/incsta/2019/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn
['category', '2016', '2017', '2018', '2019']
Done OCB
https://s.cafef.vn/bao-cao-tai-chinh-ngan-hang/ctg/incsta/2023/0/0/0/ket-qua-hoat-don

In [28]:
dfs2 = pd.read_csv(r'../csv/bank_financial_report_v2_1.csv')
dfs = pd.concat([dfs, dfs2]).reset_index(drop=True)
dfs.drop_duplicates(inplace=True)

In [29]:
test_df = dfs[(dfs['year'] == 2023) & (dfs['quarter'] == 0)]
assert test_df['stock_code'].nunique() == len(bank_stock_code), "Number of stock code is not correct"

AssertionError: Number of stock code is not correct

In [30]:
for code in bank_stock_code:
    if  code not in test_df['stock_code'].unique():
        print(f"{code} is missing")

NVB is missing


In [32]:
dfs.to_csv('../csv/bank_financial_report_v2_1.csv', index=False)

In [38]:
sheets = ['incsta', 'bsheet', 'cashflow']
sheet_types = ['income_statement' , 'balance_sheet', 'cash_flow_statement']
dfs = None
start_year = 2024
start_months = [0,2]
durations = [2,6]
count = 1

for sheet, sheet_type in zip(sheets, sheet_types):
    for start_month, duration in zip(start_months, durations):
        for stock_code in securities_stock_code:
            if count % 5 == 0:
                time.sleep(5)
            count+=1
            
            df = crawl_mix_securities(stock_code, start_year, start_month, duration, sheet)
            if dfs is None:
                dfs = df
            else:
                dfs = pd.concat([dfs, df]).reset_index(drop=True)
            print("Done", stock_code)


['category', '2020', '2021', '2022', '2023']
['category', '2016', '2017', '2018', '2019']
Done MBS
['category', '2020', '2021', '2022', '2023']
['category', '2016', '2017', '2018', '2019']
Done VND
['category', '2020', '2021', '2022', '2023']
['category', '2016', '2017', '2018', '2019']
Done SSI
['category', '2020', '2021', '2022', '2023']
['category', '2016', '2017', '2018', '2019']
Done VIX
['category', '2020', '2021', '2022', '2023']
['category', '2016', '2017', '2018', '2019']
Done ORS
['category', 'Quý  3-2023', 'Quý  4-2023', 'Quý  1-2024', 'Quý  2-2024']
['category', 'Quý  3-2022', 'Quý  4-2022', 'Quý  1-2023', 'Quý  2-2023']
['category', 'Quý  3-2021', 'Quý  4-2021', 'Quý  1-2022', 'Quý  2-2022']
['category', 'Quý  3-2020', 'Quý  4-2020', 'Quý  1-2021', 'Quý  2-2021']
['category', 'Quý  3-2019', 'Quý  4-2019', 'Quý  1-2020', 'Quý  2-2020']
['category', 'Quý  3-2018', 'Quý  4-2018', 'Quý  1-2019', 'Quý  2-2019']
Done MBS
['category', 'Quý  3-2023', 'Quý  4-2023', 'Quý  1-2024', 

In [35]:
crawl_mix_securities('VND', 2024, 0, 2, 'bsheet')

['category', '2020', '2021', '2022', '2023']
                   2020                2021                2022  \
0                   NaN                 NaN                 NaN   
1    13,218,965,740,000  36,627,504,410,000  38,670,054,790,000   
2    13,204,833,970,000  36,524,166,070,000  38,570,066,190,000   
3       973,526,890,000   2,701,561,360,000   2,556,722,530,000   
4       887,704,080,000   2,094,329,940,000   1,877,509,170,000   
..                  ...                 ...                 ...   
127  14,976,084,310,000  37,094,774,470,000  38,870,604,500,000   
128                 NaN                 NaN                 NaN   
129   2,204,301,690,000   4,349,446,870,000  12,178,440,090,000   
130                 NaN                 NaN                 NaN   
131                 NaN          10,410,000                 NaN   

                   2023 category_code  
0                   NaN           NaN  
1    40,414,384,838,078        BS_100  
2    40,278,851,863,343       

,stock_code,category_code,year,quarter,data
0,VND,BS_100,2020,0,13218965
1,VND,BS_100,2021,0,36627504
2,VND,BS_100,2022,0,38670054
3,VND,BS_100,2023,0,40414384
4,VND,BS_100,2016,0,5052018
...,...,...,...,...,...
859,VND,BS_440,2023,0,41742090
860,VND,BS_440,2016,0,5174092
861,VND,BS_440,2017,0,8051385
862,VND,BS_440,2018,0,10534641


In [39]:
test_df = dfs[(dfs['year'] == 2023) & (dfs['quarter'] == 0)]
assert test_df['stock_code'].nunique() == len(securities_stock_code), "Number of stock code is not correct"

In [ ]:
for code in securities_stock_code:
    if  code not in test_df['stock_code'].unique():
        print(f"{code} is missing")

In [40]:
dfs.to_csv('../csv/securities_financial_report_v2_1.csv', index=False)

In [30]:
import string

def replace_punctuation_with_space(sentence):
    # Create a translation table that maps all punctuation characters to a space
    translator = str.maketrans(string.punctuation , ' ' * len(string.punctuation))
    
    # Use the translation table to replace punctuation with spaces
    clean_sentence = sentence.translate(translator)
    
    # Remove any double spaces that may have been created
    clean_sentence = ' '.join(clean_sentence.split())
    
    return clean_sentence.lower()


# Example usage
original_sentence = "Hello, world! This is a test senten()ce with punctuation, hyphens-and*asterisks."
clean_sentence = replace_punctuation_with_space(original_sentence)
print(clean_sentence)

hello world this is a test senten ce with punctuation hyphens and asterisks


In [ ]:
def chop_number(x):
    x = str(x)
    if len(x) > 16:
        return int(x[:-6])
    return int(x)

def chop_mil(x):
    if abs(x) >=1_000_000:
        return int(x/1_000_000)
    return x

def fix_number_issue(df):
    df['data'] = df['data'].apply(lambda x: x.replace(",", "") if type(x) == str else x)
    df['data'] = df['data'].apply(lambda x: chop_number(x))
    df['data'] = df['data'].apply(lambda x: chop_mil(x))
    return df

## Company info

In [16]:
from vnstock3 import * 
from vnstock3 import Vnstock

Phiên bản Vnstock 0.3.0.9 đã có mặt, vui lòng cập nhật với câu lệnh : `pip install vnstock3 --upgrade`.
Lịch sử phiên bản: https://vnstocks.com/docs/tai-lieu/lich-su-phien-ban
Phiên bản hiện tại 0.3.0.8

In [17]:
symbols = bank_stock_code + non_bank_stock_code + securities_stock_code

In [18]:
len(symbols)

81

In [19]:
stock = Vnstock().stock(symbol = 'ACB', source = 'TCBS')

In [20]:
vn30 = stock.listing.symbols_by_group('VN30').values
hnx30 = stock.listing.symbols_by_group('HNX30').values

In [21]:
company_overviews = []
# Iterate through each symbol and print company information
for symbol in symbols:
    company = Vnstock().stock(symbol=symbol, source='TCBS').company
    company_overview = company.overview()
    company_overview['symbol'] = symbol
    company_overviews.append(company_overview)

df_company_overviews = pd.concat(company_overviews, ignore_index=True)
# Display the concatenated DataFrame
df_company_overviews.head()

,exchange,industry,company_type,no_shareholders,foreign_percent,outstanding_share,issue_share,established_year,no_employees,stock_rating,delta_in_week,delta_in_month,delta_in_year,short_name,website,industry_id,industry_id_v2,symbol
0,HOSE,Ngân hàng,NH,27603,0.170,5700.4,5700.4,1993,29228,2.8,0.004,-0.014,0.118,BIDV,https://www.bidv.com.vn,289,8355,BID
1,HOSE,Ngân hàng,NH,25371,0.032,1862.7,1868.8,1992,6226,2.5,-0.059,0.071,0.048,Eximbank,https://eximbank.com.vn,289,8355,EIB
2,HOSE,Ngân hàng,NH,20103,0.199,2465.8,2465.8,1996,6904,2.3,-0.024,-0.053,-0.171,Ngân hàng Phương Đông,https://www.ocb.com.vn,289,8355,OCB
3,HOSE,Ngân hàng,NH,53928,0.266,5370.0,5370.0,1988,24698,3.2,0.027,-0.011,0.216,VietinBank,http://www.vietinbank.vn,289,8355,CTG
4,HOSE,Ngân hàng,NH,25183,0.233,5589.1,5589.1,2008,23992,3.2,0.017,0.042,-0.143,Vietcombank,https://vietcombank.com.vn,289,8355,VCB


In [ ]:
mask_vn30 = df_company_overviews['symbol'].isin(vn30)
mask_hnx30 = df_company_overviews['symbol'].isin(hnx30)

df_company_overviews['stock_indices'] = 'OTHER'
df_company_overviews.loc[mask_vn30, 'stock_indices'] = 'VN30'
df_company_overviews.loc[mask_hnx30, 'stock_indices'] = 'HNX30'

In [23]:
industry_trans = {
    "Ngân hàng": "Banking",
    "Bất động sản": "Real Estate",
    "Xây dựng và Vật liệu": "Construction and Materials",
    "Thực phẩm và đồ uống": "Food and Beverages",
    "Hóa chất": "Chemicals",
    "Dịch vụ tài chính": "Financial Services",
    "Tài nguyên Cơ bản": "Basic Resources",
    "Bán lẻ": "Retail",
    "Dầu khí": "Oil and Gas",
    "Điện, nước & xăng dầu khí đốt": "Utilities (Electricity, Water & Gas)",
    "Hàng cá nhân & Gia dụng": "Personal and Household Goods",
    "Công nghệ Thông tin": "Information Technology",
    "Hàng & Dịch vụ Công nghiệp": "Industrial Goods and Services",
    "Truyền thông": "Media",
    "Du lịch và Giải trí": "Travel and Leisure",
    "Y tế": "Healthcare",
    "Ô tô và phụ tùng": "Automobiles and Parts",
    "Viễn thông": "Telecommunications"
}


In [24]:
df_company_overviews['industry'] = df_company_overviews['industry'].map(industry_trans)

In [25]:
df_company_overviews.drop(columns = ['company_type','outstanding_share','established_year','no_employees','delta_in_week','delta_in_month','delta_in_year','short_name','industry_id','industry_id_v2'], inplace = True)
df_company_overviews['issue_share'] *= 1_000_000
df_company_overviews.head()

,exchange,industry,no_shareholders,foreign_percent,issue_share,stock_rating,website,symbol,bucket
0,HOSE,Banking,27603,0.170,5.700400e+09,2.8,https://www.bidv.com.vn,BID,VN30
1,HOSE,Banking,25371,0.032,1.868800e+09,2.5,https://eximbank.com.vn,EIB,OTHER
2,HOSE,Banking,20103,0.199,2.465800e+09,2.3,https://www.ocb.com.vn,OCB,OTHER
3,HOSE,Banking,53928,0.266,5.370000e+09,3.2,http://www.vietinbank.vn,CTG,VN30
4,HOSE,Banking,25183,0.233,5.589100e+09,3.2,https://vietcombank.com.vn,VCB,VN30


In [26]:
company_profiles = []
count = 1
for symbol in symbols:
    company = Vnstock().stock(symbol=symbol, source='TCBS').company
    company_profile = company.profile()
    company_profile['symbol'] = symbol
    company_profiles.append(company_profile)
    if count % 10 == 0:
        time.sleep(10)
    count+=1

df_company_profiles = pd.concat(company_profiles, ignore_index=True)

# Combine all features into a single column with the specified format
df_company_profiles['combine_profile'] = df_company_profiles.apply(
    lambda row: '\n '.join([f"{col}: {row[col]}" for col in df_company_profiles.columns if col != 'symbol']), axis=1
)
df_company_profiles.drop(columns = ['company_profile','company_profile','history_dev','company_promise','business_risk','key_developments','business_strategies'], inplace = True)

# Display the updated DataFrame
df_company_profiles.head()


,company_name,symbol,combine_profile
0,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,BID,company_name: Ngân hàng Thương mại Cổ phần Đầu...
1,Ngân hàng Thương mại Cổ phần Xuất nhập khẩu Vi...,EIB,company_name: Ngân hàng Thương mại Cổ phần Xuấ...
2,Ngân hàng Thương mại Cổ phần Phương Đông,OCB,company_name: Ngân hàng Thương mại Cổ phần Phư...
3,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,CTG,company_name: Ngân hàng Thương mại Cổ phần Côn...
4,Ngân hàng Thương mại Cổ phần Ngoại thương Việt...,VCB,company_name: Ngân hàng Thương mại Cổ phần Ngo...


In [27]:
df_company_info = pd.merge(df_company_overviews, df_company_profiles, on='symbol')
df_company_info.head()


,exchange,industry,no_shareholders,foreign_percent,issue_share,stock_rating,website,symbol,bucket,company_name,combine_profile
0,HOSE,Banking,27603,0.170,5.700400e+09,2.8,https://www.bidv.com.vn,BID,VN30,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,company_name: Ngân hàng Thương mại Cổ phần Đầu...
1,HOSE,Banking,25371,0.032,1.868800e+09,2.5,https://eximbank.com.vn,EIB,OTHER,Ngân hàng Thương mại Cổ phần Xuất nhập khẩu Vi...,company_name: Ngân hàng Thương mại Cổ phần Xuấ...
2,HOSE,Banking,20103,0.199,2.465800e+09,2.3,https://www.ocb.com.vn,OCB,OTHER,Ngân hàng Thương mại Cổ phần Phương Đông,company_name: Ngân hàng Thương mại Cổ phần Phư...
3,HOSE,Banking,53928,0.266,5.370000e+09,3.2,http://www.vietinbank.vn,CTG,VN30,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,company_name: Ngân hàng Thương mại Cổ phần Côn...
4,HOSE,Banking,53928,0.266,5.370000e+09,3.2,http://www.vietinbank.vn,CTG,VN30,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,company_name: Ngân hàng Thương mại Cổ phần Côn...


In [28]:
df_company_info['is_bank'] = np.where(df_company_info['symbol'].isin(bank_stock_code), True, False)
df_company_info['is_securities'] = np.where(df_company_info['symbol'].isin(securities_stock_code), True, False)
df_company_info

,exchange,industry,no_shareholders,foreign_percent,issue_share,stock_rating,website,symbol,bucket,company_name,combine_profile,is_bank,is_securities
0,HOSE,Banking,27603,0.170,5.700400e+09,2.8,https://www.bidv.com.vn,BID,VN30,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,company_name: Ngân hàng Thương mại Cổ phần Đầu...,True,False
1,HOSE,Banking,25371,0.032,1.868800e+09,2.5,https://eximbank.com.vn,EIB,OTHER,Ngân hàng Thương mại Cổ phần Xuất nhập khẩu Vi...,company_name: Ngân hàng Thương mại Cổ phần Xuấ...,True,False
2,HOSE,Banking,20103,0.199,2.465800e+09,2.3,https://www.ocb.com.vn,OCB,OTHER,Ngân hàng Thương mại Cổ phần Phương Đông,company_name: Ngân hàng Thương mại Cổ phần Phư...,True,False
3,HOSE,Banking,53928,0.266,5.370000e+09,3.2,http://www.vietinbank.vn,CTG,VN30,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,company_name: Ngân hàng Thương mại Cổ phần Côn...,True,False
4,HOSE,Banking,53928,0.266,5.370000e+09,3.2,http://www.vietinbank.vn,CTG,VN30,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,company_name: Ngân hàng Thương mại Cổ phần Côn...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,HNX,Financial Services,17654,0.027,5.471000e+08,2.4,http://mbs.com.vn,MBS,HNX30,Công ty Cổ phần Chứng khoán MB,company_name: Công ty Cổ phần Chứng khoán MB\n...,False,True
79,HOSE,Financial Services,63488,0.122,1.522300e+09,2.5,http://www.vndirect.com.vn,VND,OTHER,Công ty Cổ phần Chứng khoán VNDIRECT,company_name: Công ty Cổ phần Chứng khoán VNDI...,False,True
80,HOSE,Financial Services,74489,0.425,1.813000e+09,2.3,https://www.ssi.com.vn,SSI,VN30,Công ty Cổ phần Chứng khoán SSI,company_name: Công ty Cổ phần Chứng khoán SSI\...,False,True
81,HOSE,Financial Services,45044,0.047,1.458500e+09,2.7,https://www.vixs.vn,VIX,OTHER,Công ty Cổ phần Chứng khoán VIX,company_name: Công ty Cổ phần Chứng khoán VIX\...,False,True


In [29]:
name = stock.listing.symbols_by_exchange()
name

,symbol,id,type,exchange,en_organ_name,en_organ_short_name,organ_short_name,organ_name
0,YTC,8425620,STOCK,UPCOM,Ho Chi Minh City Medical Import Export Joint S...,YTECO,XNK Y tế TP.HCM,Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố ...
1,YEG,8424579,STOCK,HSX,Yeah1 Group Corporation,Yeah1 Group,Tập đoàn Yeah1,Công ty Cổ phần Tập đoàn Yeah1
2,YBM,8424470,STOCK,HSX,Yen Bai Industry Mineral Joint Stock Company,Yen Bai Industry Mineral,Khoáng sản CN Yên Bái,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
3,YBC,8425710,STOCK,UPCOM,YenBai Cement and Minerals Joint Stock Company,YenBai Cement and Minerals,Xi măng và Khoáng sản Yên Bái,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
4,XPH,8425864,STOCK,UPCOM,Hanoi Soap Joint Stock Company,Hanoi Soap,Xà phòng Hà Nội,Công ty Cổ phần Xà phòng Hà Nội
...,...,...,...,...,...,...,...,...
2558,AAS,8425599,STOCK,UPCOM,Smart Invest Securities Joint Stock Company,SmartInvest Securities,Chứng khoán SmartInvest,Công ty Cổ phần Chứng khoán SmartInvest
2559,AAM,8424957,STOCK,HSX,Mekong Fisheries Joint Stock Company,Mekong Fisheries,Thủy sản Mekong,Công ty Cổ Phần Thủy Sản MeKong
2560,AAH,13216673,STOCK,UPCOM,Hop Nhat Joint Stock Company,Hop Nhat Coal,Than Hợp Nhất,Công ty Cổ phần Hợp Nhất
2561,AAA,8424464,STOCK,HSX,An Phat Bioplastics Joint Stock Company,An Phat Bioplastics,An Phát Bioplastics,Công ty Cổ phần Nhựa An Phát Xanh


In [30]:
df_company_info = pd.merge(df_company_info, name[['symbol', 'en_organ_name', 'en_organ_short_name', 'organ_short_name']], on='symbol', how='left')
df_company_info

,exchange,industry,no_shareholders,foreign_percent,issue_share,stock_rating,website,symbol,bucket,company_name,combine_profile,is_bank,is_securities,en_organ_name,en_organ_short_name,organ_short_name
0,HOSE,Banking,27603,0.170,5.700400e+09,2.8,https://www.bidv.com.vn,BID,VN30,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,company_name: Ngân hàng Thương mại Cổ phần Đầu...,True,False,Joint Stock Commercial Bank for Investment and...,BIDV,BIDV
1,HOSE,Banking,25371,0.032,1.868800e+09,2.5,https://eximbank.com.vn,EIB,OTHER,Ngân hàng Thương mại Cổ phần Xuất nhập khẩu Vi...,company_name: Ngân hàng Thương mại Cổ phần Xuấ...,True,False,Vietnam Export Import Commercial Joint Stock Bank,Eximbank,Eximbank
2,HOSE,Banking,20103,0.199,2.465800e+09,2.3,https://www.ocb.com.vn,OCB,OTHER,Ngân hàng Thương mại Cổ phần Phương Đông,company_name: Ngân hàng Thương mại Cổ phần Phư...,True,False,Orient Commercial Joint Stock Bank,OCB Bank,Ngân hàng Phương Đông
3,HOSE,Banking,53928,0.266,5.370000e+09,3.2,http://www.vietinbank.vn,CTG,VN30,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,company_name: Ngân hàng Thương mại Cổ phần Côn...,True,False,Vietnam Joint Stock Commercial Bank For Indust...,VietinBank,VietinBank
4,HOSE,Banking,53928,0.266,5.370000e+09,3.2,http://www.vietinbank.vn,CTG,VN30,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,company_name: Ngân hàng Thương mại Cổ phần Côn...,True,False,Vietnam Joint Stock Commercial Bank For Indust...,VietinBank,VietinBank
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,HNX,Financial Services,17654,0.027,5.471000e+08,2.4,http://mbs.com.vn,MBS,HNX30,Công ty Cổ phần Chứng khoán MB,company_name: Công ty Cổ phần Chứng khoán MB\n...,False,True,MB Securities Joint Stock Company,MB Securities,Chứng khoán MB
79,HOSE,Financial Services,63488,0.122,1.522300e+09,2.5,http://www.vndirect.com.vn,VND,OTHER,Công ty Cổ phần Chứng khoán VNDIRECT,company_name: Công ty Cổ phần Chứng khoán VNDI...,False,True,VNDIRECT Securities Corporation,VNDIRECT,Chứng khoán VNDIRECT
80,HOSE,Financial Services,74489,0.425,1.813000e+09,2.3,https://www.ssi.com.vn,SSI,VN30,Công ty Cổ phần Chứng khoán SSI,company_name: Công ty Cổ phần Chứng khoán SSI\...,False,True,SSI Securities Corporation,SSI Securities,Chứng khoán SSI
81,HOSE,Financial Services,45044,0.047,1.458500e+09,2.7,https://www.vixs.vn,VIX,OTHER,Công ty Cổ phần Chứng khoán VIX,company_name: Công ty Cổ phần Chứng khoán VIX\...,False,True,VIX Securities Joint Stock Company,VIX Securities,Chứng khoán VIX


In [ ]:
df_company_info.rename(columns = {'symbol':'stock_code',  'en_organ_name':'en_company_name', 'en_organ_short_name':'en_short_name', 'organ_short_name':'short_name'}, inplace = True)
df_company_info= df_company_info[['stock_code','company_name', 'short_name','en_company_name','en_short_name','combine_profile','industry','exchange','foreign_percent','issue_share','no_shareholders','stock_rating','website', 'stock_indices', 'is_bank', 'is_securities']]
df_company_info.head()

,stock_code,company_name,short_name,en_company_name,en_short_name,combine_profile,industry,exchange,foreign_percent,issue_share,no_shareholders,stock_rating,website,bucket,is_bank,is_securities
0,BID,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,BIDV,Joint Stock Commercial Bank for Investment and...,BIDV,company_name: Ngân hàng Thương mại Cổ phần Đầu...,Banking,HOSE,0.170,5.700400e+09,27603,2.8,https://www.bidv.com.vn,VN30,True,False
1,EIB,Ngân hàng Thương mại Cổ phần Xuất nhập khẩu Vi...,Eximbank,Vietnam Export Import Commercial Joint Stock Bank,Eximbank,company_name: Ngân hàng Thương mại Cổ phần Xuấ...,Banking,HOSE,0.032,1.868800e+09,25371,2.5,https://eximbank.com.vn,OTHER,True,False
2,OCB,Ngân hàng Thương mại Cổ phần Phương Đông,Ngân hàng Phương Đông,Orient Commercial Joint Stock Bank,OCB Bank,company_name: Ngân hàng Thương mại Cổ phần Phư...,Banking,HOSE,0.199,2.465800e+09,20103,2.3,https://www.ocb.com.vn,OTHER,True,False
3,CTG,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,VietinBank,Vietnam Joint Stock Commercial Bank For Indust...,VietinBank,company_name: Ngân hàng Thương mại Cổ phần Côn...,Banking,HOSE,0.266,5.370000e+09,53928,3.2,http://www.vietinbank.vn,VN30,True,False
4,CTG,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,VietinBank,Vietnam Joint Stock Commercial Bank For Indust...,VietinBank,company_name: Ngân hàng Thương mại Cổ phần Côn...,Banking,HOSE,0.266,5.370000e+09,53928,3.2,http://www.vietinbank.vn,VN30,True,False


In [ ]:
new_company = ({'stock_code': 'SCIC', 'company_name':'Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước - Công ty TNHH', 'en_company_name':'State Capital Investment and Trading Corporation - Limited Liability Company', 'short_name':'Đầu tư nhà nước', 'en_short_name':'State Fund', 'website':'http://www.scic.vn/index.php/vi/' , 'is_bank': False, 'is_securities': False})
df_company_info = pd.concat([df_company_info, pd.DataFrame(new_company, index=[0])], ignore_index=True)

new_company = ({'stock_code':'Viettel', 'company_name':'Tập đoàn Công nghiệp - Viễn thông Quân đội', 'short_name':'Viettel', 'en_company_name':'Viettel Group', 'en_short_name':'Viettel', 'website':'https://viettel.vn', 'is_bank': False, 'is_securities': False})
df_company_info = pd.concat([df_company_info, pd.DataFrame(new_company, index=[0])], ignore_index=True)

central_bank = ({'stock_code':'NHNN', 'company_name':'Ngân hàng Nhà nước Việt Nam', 'short_name':'NHNN', 'en_company_name':'State Bank of Vietnam', 'en_short_name':'Central Bank', 'website':'https://sbv.gov.vn', 'is_bank': True, 'is_securities': False})
df_company_info = pd.concat([df_company_info, pd.DataFrame(central_bank, index=[0])], ignore_index=True)

new_company = ({'stock_code':'PVN', 'company_name':'Tập đoàn Dầu khí Việt Nam', 'short_name':'Tập đoàn dầu khí', 'en_company_name':'Vietnam Oil and Gas Group', 'en_short_name':'Petrovietnam', 'website':'https://sbv.gov.vn', 'is_bank': False, 'is_securities': False})
df_company_info = pd.concat([df_company_info, pd.DataFrame(new_company, index=[0])], ignore_index=True)

df_company_info['is_bank'] = df_company_info['is_bank'].astype(bool)
df_company_info['is_securities'] = df_company_info['is_securities'].astype(bool)

df_company_info.to_csv('../csv/df_company_info.csv', index=False)

In [34]:
df_company_info.drop_duplicates(subset=['stock_code'], inplace=True)
df_company_info.to_csv('../csv/df_company_info.csv', index=False)

In [33]:
company_shareholders = []
count = 1
for symbol in symbols:
    company = Vnstock().stock(symbol=symbol, source='TCBS').company
    company_shareholder = company.shareholders()
    company_shareholder['symbol'] = symbol
    company_shareholders.append(company_shareholder)
    if count % 10 == 0:
        time.sleep(5)
    count+=1

df_company_shareholders = pd.concat(company_shareholders, ignore_index=True)

# Display the concatenated DataFrame
df_company_shareholders

,share_holder,share_own_percent,symbol
0,Ngân Hàng Nhà Nước Việt Nam,0.8099,BID
1,"KEB Hana Bank, Co., Ltd.",0.1500,BID
2,Khác,0.0028,BID
3,Công ty Cổ phần Tập đoàn Gelex,0.1000,EIB
4,Lương Thị Cẩm Tú,0.0112,EIB
...,...,...,...
685,Vũ Hồng Hạnh,0.0008,ORS
686,Nguyễn Thị Khánh Hòa,0.0005,ORS
687,Tạ Quang Lương,0.0003,ORS
688,Nguyễn Trát Minh Phương,0.0002,ORS


In [155]:
df_company_shareholders['share_holder'].value_counts()[:10]

share_holder
Khác                                                             81
Tập đoàn Dầu khí Việt Nam                                         6
Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước - Công ty TNHH     5
Ngân Hàng Nhà Nước Việt Nam                                       4
Nguyễn Thị Thanh Hà                                               4
PYN Elite Fund                                                    3
Tập đoàn Công nghiệp - Viễn thông Quân đội                        3
Nguyễn Văn Tuấn                                                   2
Phillip Securities Public Company Limited                         2
Nguyễn Mạnh Tuấn                                                  2
Name: count, dtype: int64

In [36]:
from openai import OpenAI
import os 
from dotenv import load_dotenv

load_dotenv()


client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
      
   return client.embeddings.create(input = text, model=model).data[0].embedding

In [35]:
import numpy as np
import pandas as pd

df_company_info = pd.read_csv('../csv/df_company_info.csv')

In [8]:
df_company_info1 = df_company_info[['stock_code', 'company_name']]
df_company_info2 = df_company_info[['stock_code', 'en_company_name']]
df_company_info2.rename(columns = {'en_company_name':'company_name'}, inplace = True)

df_company_info3 = df_company_info[['stock_code', 'en_short_name']]
df_company_info3.rename(columns = {'en_short_name':'company_name'}, inplace = True)

df_company_info4 = df_company_info[['stock_code', 'short_name']]
df_company_info4.rename(columns = {'short_name':'company_name'}, inplace = True)


df_company_name = pd.concat([df_company_info1, df_company_info2, df_company_info3, df_company_info4], ignore_index=True)
df_company_name.drop_duplicates(inplace=True)
df_company_name

C:\Users\Admin\AppData\Local\Temp\ipykernel_12324\1130616141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_company_info2.rename(columns = {'en_company_name':'company_name'}, inplace = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_12324\1130616141.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_company_info3.rename(columns = {'en_short_name':'company_name'}, inplace = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_12324\1130616141.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,stock_code,company_name
0,BID,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...
1,EIB,Ngân hàng Thương mại Cổ phần Xuất nhập khẩu Vi...
2,OCB,Ngân hàng Thương mại Cổ phần Phương Đông
3,CTG,Ngân hàng Thương mại Cổ phần Công thương Việt Nam
7,VCB,Ngân hàng Thương mại Cổ phần Ngoại thương Việt...
...,...,...
342,VIX,Chứng khoán VIX
343,ORS,Chứng khoán Tiên Phong
344,SCIC,Đầu tư nhà nước
346,NHNN,NHNN


In [9]:
name_embedding = []
for organ_name in df_company_name['company_name']:
    name_embedding.append(get_embedding(organ_name))
    
name_embedding = np.array(name_embedding)
name_embedding /= np.linalg.norm(name_embedding, axis=1)[:, np.newaxis]
name_embedding.shape

(311, 1536)

In [10]:
np.save('name_embedding.npy', name_embedding)

In [32]:
import numpy as np
import concurrent.futures

# Function to get embeddings in parallel
def get_embeddings_parallel(company_name, get_embedding):
    organ_name_embedding = []
    
    # Define a function to handle the parallel execution
    def fetch_embedding(organ_name):
        print(f"Fetching embedding for {organ_name}")
        return get_embedding(organ_name)
    
    # Use ThreadPoolExecutor to parallelize API calls
    with concurrent.futures.ThreadPoolExecutor() as executor:
        organ_name_embedding = list(executor.map(fetch_embedding, company_name))
    
    # Convert the list of embeddings to a numpy array
    organ_name_embedding = np.array(organ_name_embedding)
    
    # Normalize the embeddings
    organ_name_embedding /= np.linalg.norm(organ_name_embedding, axis=1)[:, np.newaxis]
    
    return organ_name_embedding

# Call the function and get the shape
name_embedding = get_embeddings_parallel(df_company_name['company_name'], get_embedding)
print(name_embedding.shape)

In [54]:
def find_suitable_company(name, company_name, name_embedding, threshold = 0.85):
    embedding = get_embedding(name)
    assert len(embedding) == name_embedding.shape[1], "Embedding shape is not correct"
    
    embedding = np.array(embedding)
    embedding /= np.linalg.norm(embedding)
    embedding = embedding.reshape(1, -1).T
    
    similarities = np.dot( name_embedding, embedding)
    
    if similarities.max() < threshold:
        return np.nan
    print(company_name[np.argmax(similarities)])
    return company_name[np.argmax(similarities)]

In [55]:
def find_suitable_company_parallel(company_name, name_embedding_text, name_embedding, threshold = 0.85):
    organ_name_embedding = []
    
    def find_suitable_company2(organ_name):
        return find_suitable_company(organ_name, name_embedding_text, name_embedding, threshold)
    
    # Use ThreadPoolExecutor to parallelize API calls
    with concurrent.futures.ThreadPoolExecutor() as executor:
        organ_name_embedding = list(executor.map(find_suitable_company2, company_name))
    
    # Convert the list of embeddings to a numpy array
    
    return np.array(organ_name_embedding)

In [12]:
find_suitable_company('Tập đoàn Vingroup')

VIC -> Tập đoàn Vingroup - Công ty CP


'VIC'

In [23]:
df_company_shareholders['share_holder_code'] = df_company_shareholders['share_holder'].apply(lambda x: find_suitable_company(x, df_company_name['stock_code'], name_embedding))

NHNN -> Ngân hàng Nhà nước Việt Nam
NHNN -> Ngân hàng Nhà nước Việt Nam
NHNN -> Ngân hàng Nhà nước Việt Nam
Viettel -> Tập đoàn Công nghiệp - Viễn thông Quân đội
SCIC -> Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước - Công ty TNHH
FPT -> Công ty Cổ phần FPT
MSN -> Công ty Cổ phần Tập đoàn Masan
NHNN -> Ngân hàng Nhà nước Việt Nam
SCIC -> Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước - Công ty TNHH
VIC -> Tập đoàn Vingroup - Công ty CP
SCIC -> Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước - Công ty TNHH
MSN -> Công ty Cổ phần Tập đoàn Masan
PVN -> Tập đoàn Dầu khí Việt Nam
VIC -> Tập đoàn Vingroup - Công ty CP
SCIC -> Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước - Công ty TNHH
PVN -> Tập đoàn Dầu khí Việt Nam
Viettel -> Tập đoàn Công nghiệp - Viễn thông Quân đội
PVN -> Tập đoàn Dầu khí Việt Nam
FPT -> Công ty Cổ phần FPT
PVN -> Tập đoàn Dầu khí Việt Nam
Viettel -> Tập đoàn Công nghiệp - Viễn thông Quân đội
PVN -> Tập đoàn Dầu khí Việt Nam
SCIC -> Tổng Công ty Đầu Tư Và Kinh Doanh V

In [25]:
df_company_shareholders

,share_holder,share_own_percent,symbol,share_holder_code
0,Ngân Hàng Nhà Nước Việt Nam,0.8099,BID,NHNN
1,"KEB Hana Bank, Co., Ltd.",0.1500,BID,NaN
2,Khác,0.0028,BID,NaN
3,Công ty Cổ phần Tập đoàn Gelex,0.1000,EIB,NaN
4,Lương Thị Cẩm Tú,0.0112,EIB,NaN
...,...,...,...,...
685,Vũ Hồng Hạnh,0.0008,ORS,NaN
686,Nguyễn Thị Khánh Hòa,0.0005,ORS,NaN
687,Tạ Quang Lương,0.0003,ORS,NaN
688,Nguyễn Trát Minh Phương,0.0002,ORS,NaN


In [26]:
df_company_shareholders2 = df_company_shareholders.dropna(subset=['share_holder_code'])
df_company_shareholders2

,share_holder,share_own_percent,symbol,share_holder_code
0,Ngân Hàng Nhà Nước Việt Nam,0.8099,BID,NHNN
23,Ngân Hàng Nhà Nước Việt Nam,0.6446,CTG,NHNN
26,Ngân Hàng Nhà Nước Việt Nam,0.7480,VCB,NHNN
39,Tập đoàn Công nghiệp - Viễn thông Quân đội,0.1476,MBB,Viettel
40,Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước...,0.0986,MBB,SCIC
59,Công ty Cổ phần FPT,0.0677,TPB,FPT
89,Công ty Cổ phần Tập đoàn Masan,0.1488,TCB,MSN
119,Ngân Hàng Nhà Nước Việt Nam,0.6446,CTG,NHNN
172,Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước...,0.3600,VNM,SCIC
192,Tập đoàn Vingroup - Công ty CP,0.6934,VHM,VIC


In [27]:
df_company_shareholders2.drop(columns=['share_holder'], inplace=True)
df_company_shareholders2.rename(columns = {'share_holder_code':'stock_code', 'symbol':'invest_on', 'share_own_percent':'percent' }, inplace = True)
df_company_shareholders2


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,share_own_percent,invest_on,stock_code
0,0.8099,BID,NHNN
23,0.6446,CTG,NHNN
26,0.7480,VCB,NHNN
39,0.1476,MBB,Viettel
40,0.0986,MBB,SCIC
59,0.0677,TPB,FPT
89,0.1488,TCB,MSN
119,0.6446,CTG,NHNN
172,0.3600,VNM,SCIC
192,0.6934,VHM,VIC


In [28]:
company_subsidiaries = []

for symbol in symbols:
    company = Vnstock().stock(symbol=symbol, source='TCBS').company
    company_subsidiary = company.subsidiaries()
    company_subsidiary['symbol'] = symbol
    company_subsidiaries.append(company_subsidiary)

df_company_subsidiaries = pd.concat(company_subsidiaries, ignore_index=True)

# Display the concatenated DataFrame
df_company_subsidiaries

,sub_company_name,sub_own_percent,symbol
0,Công ty TNHH Đầu tư Phát triển Quốc tế,1.000,BID
1,Công ty TNHH MTV Quản Lý Nợ Và Khai Thác Tài S...,1.000,BID
2,Ngân Hàng Thương Mại Cổ Phần Đầu Tư Và Phát Tr...,1.000,BID
3,Công ty Chứng Khoán Campuchia - Việt Nam,0.985,BID
4,Ngân Hàng Đầu Tư Và Phát Triển Campuchia,0.985,BID
...,...,...,...
1420,Tổng Công ty Cổ phần Đường Sông Miền Nam,0.000,VIX
1421,Công ty cổ phần Tập đoàn Xây dựng Tracodi,0.047,ORS
1422,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,0.000,ORS
1423,Công ty Cổ phần Dịch Vụ Xuất Khẩu Lao Động Và ...,0.000,ORS


In [33]:
listing_name = stock.listing.all_symbols()
listing_name_em = get_embeddings_parallel(listing_name['organ_name'], get_embedding)
listing_name_em.shape

Fetching embedding for Công ty Cổ phần 32
Fetching embedding for Công ty Cổ phần Nhựa An Phát Xanh
Fetching embedding for Công ty Cổ Phần Thủy Sản MeKong
Fetching embedding for Công ty Cổ phần Chứng khoán SmartInvest
Fetching embedding for Công ty Cổ phần Tập đoàn Tiên Sơn Thanh Hóa
Fetching embedding for Công ty Cổ phần AAV Group
Fetching embedding for Ngân hàng Thương mại Cổ phần An Bình
Fetching embedding for Công ty Cổ phần Truyền thông VMG
Fetching embedding for Công ty Cổ phần Bảo hiểm Ngân hàng Nông Nghiệp
Fetching embedding for Công ty Cổ phần Đầu tư Nhãn Hiệu Việt
Fetching embedding for Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận
Fetching embedding for Công ty Cổ phần Xuất nhập khẩu Thủy sản Bến Tre
Fetching embedding for Công ty Cổ phần Chứng khoán An Bình
Fetching embedding for Ngân hàng Thương mại Cổ phần Á Châu
Fetching embedding for Công ty Cổ phần Đầu tư và Xây dựng Bình Dương ACC
Fetching embedding for Công ty Cổ phần Bê tông ly tâm An Giang
Fetching embedding for Cô

(1598, 1536)

In [34]:
np.save('listing_name_em.npy', listing_name_em)

In [52]:
df_company_subsidiaries

,sub_company_name,sub_own_percent,symbol,subsidiary_code
0,Công ty TNHH Đầu tư Phát triển Quốc tế,1.000,BID,nan
1,Công ty TNHH MTV Quản Lý Nợ Và Khai Thác Tài S...,1.000,BID,nan
2,Ngân Hàng Thương Mại Cổ Phần Đầu Tư Và Phát Tr...,1.000,BID,nan
3,Công ty Chứng Khoán Campuchia - Việt Nam,0.985,BID,nan
4,Ngân Hàng Đầu Tư Và Phát Triển Campuchia,0.985,BID,nan
...,...,...,...,...
1420,Tổng Công ty Cổ phần Đường Sông Miền Nam,0.000,VIX,nan
1421,Công ty cổ phần Tập đoàn Xây dựng Tracodi,0.047,ORS,nan
1422,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,0.000,ORS,CTG
1423,Công ty Cổ phần Dịch Vụ Xuất Khẩu Lao Động Và ...,0.000,ORS,nan


In [ ]:
# # df_company_subsidiaries['subsidiary_code'] = find_suitable_company_parallel(df_company_subsidiaries['sub_company_name'], listing_name['ticker'], listing_name_em, threshold = 0.9)

# df_company_subsidiaries['subsidiary_code'] = df_company_subsidiaries['sub_company_name'].apply(lambda x: find_suitable_company(x, listing_name['ticker'], listing_name_em, threshold = 0.9))

In [60]:
df_company_subsidiaries

,sub_company_name,sub_own_percent,symbol,subsidiary_code
0,Công ty TNHH Đầu tư Phát triển Quốc tế,1.000,BID,nan
1,Công ty TNHH MTV Quản Lý Nợ Và Khai Thác Tài S...,1.000,BID,nan
2,Ngân Hàng Thương Mại Cổ Phần Đầu Tư Và Phát Tr...,1.000,BID,nan
3,Công ty Chứng Khoán Campuchia - Việt Nam,0.985,BID,nan
4,Ngân Hàng Đầu Tư Và Phát Triển Campuchia,0.985,BID,nan
...,...,...,...,...
1420,Tổng Công ty Cổ phần Đường Sông Miền Nam,0.000,VIX,SWC
1421,Công ty cổ phần Tập đoàn Xây dựng Tracodi,0.047,ORS,nan
1422,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,0.000,ORS,CTG
1423,Công ty Cổ phần Dịch Vụ Xuất Khẩu Lao Động Và ...,0.000,ORS,nan


In [62]:
original_sub = df_company_subsidiaries['sub_company_name'].copy()
organ_name_to_ticker_ = dict(zip(listing_name['organ_name'], listing_name['ticker']))
df_company_subsidiaries['sub_company_name'] = df_company_subsidiaries['sub_company_name'].map(organ_name_to_ticker_).fillna(df_company_subsidiaries['sub_company_name'])

changed_value_sub = df_company_subsidiaries[df_company_subsidiaries['sub_company_name'] != original_sub]
changed_value_sub.rename(columns = {'sub_company_name':'invest_on', 'sub_own_percent':'percent', 'symbol':'stock_code'}, inplace = True)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [68]:
changed_value_sub

,invest_on,percent,stock_code
8,BSI,0.520,BID
9,BIC,0.510,BID
20,ND2,0.052,BID
21,PVY,0.040,BID
22,DPM,0.037,BID
...,...,...,...
1418,VTX,0.000,VIX
1419,VHL,0.000,VIX
1420,SWC,0.000,VIX
1422,CTG,0.000,ORS


In [69]:
df_sub_and_shareholders = pd.concat([df_company_shareholders2, changed_value_sub], ignore_index=True)
df_sub_and_shareholders

,percent,invest_on,stock_code
0,0.8099,BID,NHNN
1,0.6446,CTG,NHNN
2,0.7480,VCB,NHNN
3,0.1476,MBB,Viettel
4,0.0986,MBB,SCIC
...,...,...,...
377,0.0000,VTX,VIX
378,0.0000,VHL,VIX
379,0.0000,SWC,VIX
380,0.0000,CTG,ORS


In [71]:
df_sub_and_shareholders.drop_duplicates(inplace=True)
df_sub_and_shareholders2 = df_sub_and_shareholders[df_sub_and_shareholders['stock_code'] != df_sub_and_shareholders['invest_on']]
df_sub_and_shareholders2


,percent,invest_on,stock_code
0,0.8099,BID,NHNN
1,0.6446,CTG,NHNN
2,0.7480,VCB,NHNN
3,0.1476,MBB,Viettel
4,0.0986,MBB,SCIC
...,...,...,...
377,0.0000,VTX,VIX
378,0.0000,VHL,VIX
379,0.0000,SWC,VIX
380,0.0000,CTG,ORS


In [72]:
df_sub_and_shareholders2.to_csv('../csv/df_sub_and_shareholders.csv', index=False)